In [ ]:
import numpy as np
import pandas as pd
import time
import datetime
import ast
import sqlite3
from DeepCCS.model.encoders import SmilesToOneHotEncoder
from DeepCCS.model.splitter import SMILESsplitter

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import keras
from keras.optimizers import adam, rmsprop
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Input, Concatenate
from keras.layers import Conv1D, MaxPooling1D,  Activation, BatchNormalization, Flatten
from keras import backend as K

In [ ]:
HMDB_SQLITE_FILE = "hmdb_metabolites.sql"
datafile = "DATASETS.h5"

In [ ]:
from sklearn.metrics import median_absolute_error, mean_absolute_error, r2_score, mean_squared_error
def relative_mean(Y_true, Y_pred):
    mean = np.mean((abs(Y_pred - Y_true) / Y_true) * 100)
    return mean

def relative_median(Y_true, Y_pred):
    med = np.median((abs(Y_pred - Y_true) / Y_true) * 100)
    return med

def percentile_95(Y_true, Y_pred):
    percentile = np.percentile((abs(Y_pred - Y_true) / Y_true) * 100, 95)
    return percentile

def percentile_90(Y_true, Y_pred):
    percentile = np.percentile((abs(Y_pred - Y_true) / Y_true) * 100, 90)
    return percentile

We use an sqlite version of the original HMDB.xml file. It's easier to parse and re-use. Parsing script is HMDB_sql_converter.py

In [ ]:
sql_connection = sqlite3.connect(HMDB_SQLITE_FILE)
cursor = sql_connection.cursor()
SMILES = []
polar_surface_area = []
logS = []
refractivity = []
polarizability = []
logP_alogps = []
logP_chemaxon = []
query = "SELECT DISTINCT SMILES, polar_surface_area, logS, refractivity, polarizability, \
logP_ALOGPS, logP_ChemAxon \
FROM HMDB where SMILES is not null and \
polar_surface_area is not null and \
logS is not null and \
refractivity is not null and \
polarizability is not null and \
logP_ALOGPS is not null and \
logP_ChemAxon is not null;"
for i in cursor.execute(query):
    SMILES.append(i[0])
    polar_surface_area.append(float(i[1]))
    logS.append(float(i[2]))
    refractivity.append(float(i[3]))
    polarizability.append(float(i[4]))
    logP_alogps.append(float(i[5]))
    logP_chemaxon.append(float(i[6]))

We now filter the dataset to keep only SMILES with an appropriate length.

In [ ]:
Y = [np.array(polar_surface_area), np.array(logS), np.array(refractivity), np.array(polarizability),
     np.array(logP_alogps), np.array(logP_chemaxon)]
X = np.array(SMILES)

# Filter SMILES by length
smiles_splitter_multi = SMILESsplitter()
lengths = [len(smiles_splitter_multi.split(x)) for x in X]
lengths_filter = np.array(lengths) <= 250

X = X[lengths_filter.astype(bool)]
Y = [target[lengths_filter.astype(bool)] for target in Y]

The objective is to measure the impact of learning the internal representation using a multi-output problem and to re-use the internal representation to predict CCS. To make sure the SMILES encoder will be compatible with both problems, we will train a SMILES encoder that is compatible with both datasets.

In [ ]:
smiles_encoder_ccs = SmilesToOneHotEncoder()
smiles_encoder_ccs.load_encoder("SMILES_encoder.json")

smiles_encoder_multi = SmilesToOneHotEncoder()
smiles_encoder_multi.fit(X)

all_symbols = smiles_encoder_ccs.converter.keys() + smiles_encoder_multi.converter.keys()

smiles_encoder_multi = SmilesToOneHotEncoder()
for i, j in enumerate(set(all_symbols)):
    smiles_encoder_multi.converter[j] = i
smiles_encoder_multi._is_fit = True

print("There is {} symbols in the SMILES encoder".format(len(smiles_encoder_multi.converter)))

We now have a SMILES encoder, a bunch of SMILES and a series of properties to predict.
We have to seperate the data between a train, valid and test set.

In [ ]:
# Split train - test
np.random.seed(432)
mask_train = np.zeros(len(X), dtype=int)
mask_train[:int(len(X) * 0.7)] = 1
np.random.shuffle(mask_train)
mask_test = 1 - mask_train
mask_train = mask_train.astype(bool)
mask_test = mask_test.astype(bool)

X_pooled = X[mask_train]
X_test = X[mask_test]

Y_pooled = [i[mask_train] for i in Y]
Y_test = [i[mask_test] for i in Y]

# Split train - valid
mask_train = np.zeros(len(X_pooled), dtype=int)
mask_train[:int(len(X_pooled) * 0.9)] = 1
np.random.shuffle(mask_train)
mask_valid = 1 - mask_train
mask_train = mask_train.astype(bool)
mask_valid = mask_valid.astype(bool)

X_train = X_pooled[mask_train]
X_valid = X_pooled[mask_valid]

Y_train = [i[mask_train] for i in Y_pooled]
Y_valid = [i[mask_valid] for i in Y_pooled]

We have to encode the SMILES with our SMILES encoder.

In [ ]:
# # SMILES encoding ----> To execute only if you have enough RAM power (in general, not a standard PC RAM)
# X_train_encoded = smiles_encoder_multi.transform(X_train)
# X_valid_encoded = smiles_encoder_multi.transform(X_valid)
# X_test_encoded = smiles_encoder_multi.transform(X_test)
# smiles_encoder_multi.save_encoder("Smiles_encoder_multi.json")

We now implement the network's structure

In [ ]:
# Network structure
smile_input_layer = Input(shape=(250, len(smiles_encoder_multi.converter)), name="smiles")
conv = Conv1D(64, kernel_size=4, activation='relu', kernel_initializer='normal')(smile_input_layer)

previous = conv
for i in range(6):
    conv = Conv1D(64, kernel_size=4, activation='relu', kernel_initializer='normal')(previous)
    if i == 5:
        pool = MaxPooling1D(pool_size=2, strides=2)(conv)
    else:
        pool = MaxPooling1D(pool_size=2, strides=1)(conv)
    previous = pool

flat = Flatten()(previous)

# polar_surface_area
previous = flat
for i in range(2):
    dense_layer = Dense(384, activation="relu", kernel_initializer='normal')(previous)
    previous = dense_layer
output_logp = Dense(1, activation="linear", name="polar_surface_area")(previous)

# logS
previous = flat
for i in range(2):
    dense_layer = Dense(384, activation="relu", kernel_initializer='normal')(previous)
    previous = dense_layer
output_logs = Dense(1, activation="linear", name="logs")(previous)

# refractivity

previous = flat
for i in range(2):
    dense_layer = Dense(384, activation="relu", kernel_initializer='normal')(previous)
    previous = dense_layer
output_refractivity = Dense(1, activation="linear", name="refractivity")(previous)

# polarizability
previous = flat
for i in range(2):
    dense_layer = Dense(384, activation="relu", kernel_initializer='normal')(previous)
    previous = dense_layer
output_polarizability = Dense(1, activation="linear", name="polarizability")(previous)

#logP_alogps
previous = flat
for i in range(2):
    dense_layer = Dense(384, activation="relu", kernel_initializer='normal')(previous)
    previous = dense_layer
output_logp_alogps = Dense(1, activation="linear", name="logp_alogps")(previous)

#Logp_Chemaxon
previous = flat
for i in range(2):
    dense_layer = Dense(384, activation="relu", kernel_initializer='normal')(previous)
    previous = dense_layer
output_logp_chemaxon = Dense(1, activation="linear", name="logp_chemaxon")(previous)

# optimizer and compile
opt = getattr(keras.optimizers, 'adam')
opt = opt()
model = Model(input=smile_input_layer,
              outputs=[output_logp, output_logs, output_refractivity, output_polarizability, 
                       output_logp_alogps, output_logp_chemaxon])
model.compile(optimizer=opt, loss='mean_squared_error')

model.summary()

In [ ]:
model_file = "Model_multioutput_2018-11-28-001.model"
model.load_weights(model_file)
y_pred = model.predict(X_test_encoded)

In [ ]:
properties = ["polar_surface_area", "logS", "refractivity", "polarizability", "logP_alogps", "logP_chemaxon"]
for i, p in enumerate(properties):
    print(p)
    mse = mean_squared_error(y_true=Y_test[i], y_pred = y_pred[i])
    r2 = r2_score(y_true=Y_test[i], y_pred = y_pred[i])
    mean_abs_err = mean_absolute_error(y_true=Y_test[i], y_pred = y_pred[i])
    median_abs_err = median_absolute_error(y_true=Y_test[i], y_pred = y_pred[i])
    relative_mean_err = relative_mean(Y_test[i], y_pred[i].flatten())
    relative_median_err = relative_median(Y_test[i], y_pred[i].flatten())
    ninety = percentile_90(Y_test[i], y_pred[i].flatten())
    ninety_five = percentile_95(Y_test[i], y_pred[i].flatten())
    print("MSE: {}".format(mse))
    print("R2: {}".format(r2))
    print("median absolute: {}".format(median_abs_err))
    print("mean absolute: {}".format(mean_abs_err))
    print("median relative: {}".format(relative_median_err))
    print("mean relative: {}".format(relative_mean_err))
    print("90 %: {}".format(ninety))
    print("95 %: {}".format(ninety_five))
    print("------------------")

# Retrain model for CCS prediction
Now that the multi-output model can perform predictions for multiple properties, we will use its internal representation to perform CCS prediction. This will tell us if we have enough data to train the internal representation for CCS predictions using only the CCS data. Furthermore, it will show that CNN are appropriate for molecular descriptor predictions directly from SMILES.

In [ ]:
#Read the data exactly the same way as it was done for the first experiment
from DeepCCS.utils import read_dataset, filter_data
from DeepCCS.model.encoders import AdductToOneHotEncoder

datasets_names = ["MetCCS_train_pos", "MetCCS_train_neg", "MetCCS_test_pos", "MetCCS_test_neg", 
                  "Astarita_pos", "Astarita_neg", "Baker", "McLean", "CBM"] 

datasets = [read_dataset(datafile, d_name) for d_name in datasets_names] # Read
datasets = [filter_data(d_set) for d_set in datasets] # Filter

np.random.seed(777)
save_test_sets_data = {}
pooled_set = []
test_sets = []
train_set = []
validation_set = []
test_sets_names = []
for i, dset in enumerate(datasets_names):
    if dset in ["MetCCS_train_pos", "MetCCS_train_neg"]:
        pooled_set.append([np.array(datasets[i]["SMILES"]),
                           np.array(datasets[i]["Adducts"]),
                           np.array(datasets[i]["CCS"])])
    elif dset in ["MetCCS_test_pos", "MetCCS_test_neg", "Astarita_pos", "Astarita_neg"]:
        test_sets.append([np.array(datasets[i]["SMILES"]),
                          np.array(datasets[i]["Adducts"]),
                          np.array(datasets[i]["CCS"])])
        test_sets_names.append(dset)
    elif dset in ["Baker", "McLean", "CBM"]:
        smiles = np.array(datasets[i]["SMILES"])
        ccs = np.array(datasets[i]["CCS"])
        adducts = np.array(datasets[i]["Adducts"])
        
        # We use binary masks to split the datasets between pooled and test
        mask_pooled = np.zeros(len(smiles), dtype=int)
        mask_pooled[:int(len(smiles) * 0.8)] = 1  # The remaining 20% goes in the test set.
        np.random.shuffle(mask_pooled)
        mask_test = 1 - mask_pooled
        mask_pooled = mask_pooled.astype(bool)
        mask_test = mask_test.astype(bool)
        
        pooled_set.append([smiles[mask_pooled], adducts[mask_pooled], ccs[mask_pooled]])
        test_sets.append([smiles[mask_test], adducts[mask_test], ccs[mask_test]])
        test_sets_names.append(dset)
# Split pooled between train (90%) and validation (10%)
smiles_pooled = np.concatenate([i[0] for i in pooled_set])
adducts_pooled = np.concatenate([i[1] for i in pooled_set])
ccs_pooled = np.concatenate([i[2] for i in pooled_set])

mask_train = np.zeros(len(smiles_pooled), dtype=int)
mask_train[:int(len(smiles_pooled) * 0.9)] = 1  # The remaining 10% goes in the validation set.
np.random.shuffle(mask_train)
mask_valid = 1 - mask_train
mask_train = mask_train.astype(bool)
mask_valid = mask_valid.astype(bool)

train_set = [smiles_pooled[mask_train], adducts_pooled[mask_train], ccs_pooled[mask_train]]
validation_set = [smiles_pooled[mask_valid], adducts_pooled[mask_valid], ccs_pooled[mask_valid]]

#We use the SMILES encoder that was used for encoding the HMDB database.
train_set[0] = smiles_encoder_multi.transform(train_set[0])
validation_set[0] = smiles_encoder_multi.transform(validation_set[0])

adducts_encoder = AdductToOneHotEncoder()
adducts_encoder.fit(np.concatenate([dset["Adducts"] for dset in datasets]))
train_set[1] = adducts_encoder.transform(train_set[1])
validation_set[1] = adducts_encoder.transform(validation_set[1])

In [ ]:
# Keep only the input layer and the conv. + max. pooling layers + the flatten layer
del model.layers[15:]

# Set trainable property at false to lock weights
for l in model.layers:
    l.trainable = False

    
adduct_input_layer = Input(shape=(len(adducts_encoder.converter),), name="adduct")

previous = model.layers[-1].output

remix_layer = keras.layers.concatenate([previous, adduct_input_layer], axis=-1)
previous = remix_layer

# Insert new dense layers
for i in range(2):
    dense_layer = Dense(384, activation="relu", kernel_initializer='normal')(previous)
    previous = dense_layer

#Insert new ouput layer
output = Dense(1, activation="linear")(previous)


# Compile
opt = adam(lr=0.0001)
new_model = Model(inputs=[model.layers[0].input, adduct_input_layer], outputs=output)
new_model.compile(optimizer=opt, loss='mean_squared_error')

new_model.summary()

In [ ]:
#Convert the pure keras model to as DeepCCS model (done manually)
from DeepCCS.model.DeepCCS import DeepCCSModel

deepCCS_model = DeepCCSModel()
deepCCS_model.adduct_encoder = adducts_encoder
deepCCS_model.smiles_encoder = smiles_encoder_multi
deepCCS_model.model = new_model

In [ ]:
#Add a global test set (for overall performances)
test_smiles_global = np.concatenate([t[0] for t in test_sets])
test_adducts_global = np.concatenate([t[1] for t in test_sets])
test_ccs_global = np.concatenate([t[2] for t in test_sets])
test_sets.append([test_smiles_global, test_adducts_global, test_ccs_global])
test_sets_names = test_sets_names + ["global"]

In [ ]:
model_file = "Model_CCS_after_multioutput_20180912.h5"
deepCCS_model.model.load_weights(model_file)
for i, t in enumerate(test_sets):
    predictions = deepCCS_model.predict(t[0], t[1]).flatten()
    print(test_sets_names[i])
    print("R2: {}".format(r2_score(y_true=t[2], y_pred=predictions)))
    print("Mean relative: {}".format(relative_mean(Y_true=t[2], Y_pred=predictions)))
    print("Median relative: {}".format(relative_median(Y_true=t[2], Y_pred=predictions)))
    print("-------------------------------------")